# Puxando dados fundamentalistas da CVM

In [ ]:
import pandas as pd
import requests
import os
import zipfile

In [ ]:
anos = range(2010, 2023)

url_base = f'http://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/'

os.chdir(r'C:\Users\lsiqu\dev\github\codigos_curso\Galáxia_9_ferramentas_fin\dados_cvm')

In [ ]:
for ano in anos:
        
    download = requests.get(url_base + f'DFP/DADOS/dfp_cia_aberta_{ano}.zip')

    open(f"dfp_cia_aberta_{ano}.zip", "wb").write(download.content)

In [ ]:
lista_demonstracoes_2010_2022 = []

In [ ]:
diretorio_atual = os.getcwd()

for arquivo in os.listdir(diretorio_atual):

    arquivo_zip = zipfile.ZipFile(arquivo)
           
    tipo = "dfp"
        
    ano = arquivo[-8: -4]
    
    for planilha in arquivo_zip.namelist():
        
        if planilha != f"{tipo}_cia_aberta_{ano}.csv":
                   
            demonstracao = pd.read_csv(arquivo_zip.open(planilha), sep = ";", 
                                       decimal = ",", encoding = 'ISO-8859-1', chunksize=1000)

            demonstracao = pd.concat(demonstracao, ignore_index=True)

            demonstracao["tipo_doc"] = len(demonstracao) * [tipo]
            
            lista_demonstracoes_2010_2022.append(demonstracao)

In [ ]:
base_dados = pd.concat(lista_demonstracoes_2010_2022)

base_dados

In [ ]:
base_dados[["con_ind", "tipo_dem"]] = base_dados['GRUPO_DFP'].str.split("-", expand = True)

base_dados["tipo_dem"] = base_dados["tipo_dem"].str.strip()
base_dados["con_ind"] = base_dados["con_ind"].str.strip()
base_dados["con_ind"] = base_dados["con_ind"].astype(str)

base_dados

In [ ]:
base_dados = base_dados.loc[:, ~base_dados.columns.isin(['ST_CONTA_FIXA', 'COLUNA_DF', 'GRUPO_DFP'])]

In [ ]:
base_dados = base_dados[base_dados["ORDEM_EXERC"] != "PENÚLTIMO"]

In [ ]:
#usar o ctrl F 

lista_dem = base_dados['tipo_dem'].unique()

lista_dem

In [ ]:
#usar o ctrl F 

lista_empresas = base_dados['DENOM_CIA'].unique()

lista_empresas

In [ ]:
weg_dre = base_dados[(base_dados["DENOM_CIA"] == "WEG S.A.") & 
                     (base_dados["tipo_dem"] == "Demonstração do Resultado") &
                    (base_dados["tipo_doc"] == "dfp") & 
                    (base_dados["DS_CONTA"] == "Receita de Venda de Bens e/ou Serviços") & 
                    (base_dados["con_ind"] == "DF Consolidado")] 
                                                         

weg_dre